In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import Input
from keras.models import load_model
#import tensorflow as tf
#from tensorflow import keras
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

#base_model = InceptionV3(include_top=False,
#                         input_shape=(150, 150, 3),
#                        weights = 'imagenet')
#
base_model.trainable = False

base_model.summary()

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True
                              )
#rescale=1/255.)

train_ds = train_gen.flow_from_directory('../Week8/clothing-dataset-small/train/',
                                         target_size=(300,300),                              
                                         batch_size=20)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)#rescale=1/255.)

val_ds = val_gen.flow_from_directory('../Week8/clothing-dataset-small/validation/',
                                     target_size=(300,300),
                                     batch_size=20)

In [ ]:
inputs = Input(shape=(300,300,3))
base = base_model(inputs, training=False)

#base = last_layer_out.output
#x = layers.Flatten()(base_model.output)
vectors = layers.GlobalAveragePooling2D()(base)

#x = layers.Dense(16, activation='relu')(x)
inner = layers.Dense(128, activation='relu')(vectors)
drop = layers.Dropout(0.2)(inner)
outputs = layers.Dense(10)(drop)
#model = keras.Model(inputs, outputs)

from tensorflow.keras import Model

#modelx = Model(base_model.input, x)
modelx = Model(inputs, outputs)

optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy(from_logits=True)

modelx.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [ ]:
history = modelx.fit(train_ds,
                     validation_data=val_ds,
                     steps_per_epoch=50,
                     epochs=10,
                     #validation_steps=5,
                     #verbose=2
                    )

In [ ]:
modelx.save('clothing_model.h5')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax = plt.plot(history.history['val_accuracy'], label='Val')
ax = plt.plot(history.history['accuracy'], label='Train')
    
plt.show()    

In [ ]:
img2 = load_img('pants.jpg', target_size=(300,300))

x = np.array(img2)
X = np.array([x])
X = preprocess_input(X)

pred = modelx.predict(X)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

### TF-Lite Conversion

In [ ]:
import tensorflow.lite as tflite
from keras_preprocessing.image import load_img
from keras.applications.xception import preprocess_input

In [ ]:
model = load_model('clothing_model.h5')

In [ ]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tfLiteModel = converter.convert()

with open('clothing_model.tflite', 'wb') as f:
    f.write(tfLiteModel)

In [ ]:
!dir

In [ ]:
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

inputIdx = interpreter.get_input_details()[0]['index']

outputIdx = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(inputIdx, X)

interpreter.invoke()

pred = interpreter.get_tensor(outputIdx)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

### Remove TF dependency

In [ ]:
from PIL import Image

In [ ]:
with Image.open('pants.jpg') as img:
    img = img.resize((300,300), Image.Resampling.NEAREST)
img    

In [ ]:
def preprocess_input(x):
    x /= 127.5
    x -= -1.
    return x

x = np.array(img, dtype='float32')
X = np.array([x])
X = preprocess_input(X)

In [ ]:
interpreter2 = tflite.Interpreter(model_path='clothing_model.tflite')

interpreter2.allocate_tensors()

inputIdx = interpreter2.get_input_details()[0]['index']

outputIdx = interpreter2.get_output_details()[0]['index']

interpreter2.set_tensor(inputIdx, X)

interpreter2.invoke()

preds = interpreter2.get_tensor(outputIdx)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

### Lambda Function

In [4]:
from keras_image_helper import create_preprocessor
import tflite_runtime.interpreter as tflite
#from keras_preprocessing.image import load_img

interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

inputIdx = interpreter.get_input_details()[0]['index']
outputIdx = interpreter.get_output_details()[0]['index']

preprocessor = create_preprocessor('xception', target_size=(300,300))
classes = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

# url = 'http://bit.ly/mlbookcamp-pants'

def predict(url):
    
    X = preprocessor.from_url(url)
    interpreter.set_tensor(inputIdx, X)
    interpreter.invoke()
    pred = interpreter.get_tensor(outputIdx)
    
    return dict(zip(classes, pred[0]))

In [5]:

url = 'http://bit.ly/mlbookcamp-pants'

predict(url)

[('dress', -0.9170631),
 ('hat', -3.8840833),
 ('longsleeve', -0.69299227),
 ('outwear', -0.96617234),
 ('pants', 8.59048),
 ('shirt', -1.4101405),
 ('shoes', -2.6699123),
 ('shorts', 6.206608),
 ('skirt', -0.20017798),
 ('t-shirt', -2.6417527)]